# Function calling

https://platform.openai.com/docs/guides/function-calling

Function Calling, 또는 Tool calling 은 OpenAI의 GPT 모델이 특정 작업을 수행할 수 있도록 함수 호출을 지원하는 기능이다.

LLM 의 한계( Knowledge cut-off ) 를 극복하기 위한 방안.

이 기능을 활용하면 GPT 모델이 단순한 텍스트 생성뿐만 아니라 더 복잡한 작업도 자동으로 처리할 수 있다.

Function Calling을 통해 다음과 같은 작업을 수행할 수 있다:

1. **API 호출**: 외부 API를 호출하여 실시간 데이터를 가져오거나 특정 작업을 수행할 수 있다.  
   예) 날씨 정보 조회, 금융 데이터 조회

2. **데이터 처리**: 특정 함수를 호출하여 데이터 처리 작업을 수행할 수 있다.  
   예) 텍스트 데이터 분석, 통계 계산

3. **자동화된 작업**: 여러 단계의 작업을 자동화하여 수행할 수 있다.  
   예) 사용자가 요청한 내용을 기반으로 보고서 생성, 이메일 작성

**주요 특징**

- **함수 정의**: 사전에 정의된 함수나 API 엔드포인트를 GPT 모델에 통합할 수 있다.
- **자동 호출**: 사용자가 명시적으로 요청하지 않아도, GPT 모델이 컨텍스트를 이해하고 자동으로 적절한 함수를 호출할 수 있다.
- **입출력 처리**: 함수의 입력값을 자동으로 생성하고, 함수 호출 후 반환된 결과를 적절히 처리하여 사용자에게 전달할 수 있다.

<img src="https://cdn.openai.com/API/docs/images/function-calling-diagram-steps.png" alt="Function Calling Diagram" width="600"/>

## open weather

https://home.openweathermap.org/

In [1]:
from google.colab import userdata
from openai import OpenAI
import os

# API_KEY 를 명시적으로 전달하는 방법 - 이걸 더 추천!
OPENAI_API_KEY = userdata.get('MY_OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

# 환경변수에서 API_KEY 를 전달하는 방법
# os.environ("OPENAI_API_KEY") = userdata.get("OPENAI_API_KEY")
# client = OpenAI()

OPEN_WEATHER_API_KEY = userdata.get('OPEN_WEATHER_API_KEY')

## Function 정의

In [7]:
import requests
import json

def get_current_weather(city, units="metric"):
  url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPEN_WEATHER_API_KEY}&units={units}"
  response = requests.get(url)
  data = response.json()

  weather_info = {
      "location"    : city,
      "unit"        : "celcius" if units == "metric" else "fahrenheit",
      "temperature" : "Not Found",
      "humidity"    : "Not Found",
      "description" : "Not Found"
  }

  if response.status_code == 200:
    weather_info["temperature"] = data["weather"][0]["description"]
    weather_info["humidity"]    = data["main"]["temp"]
    weather_info["description"] = data["main"]["humidity"]

  return json.dumps(weather_info)

get_current_weather("강남구", "metric")

'{"location": "\\uac15\\ub0a8\\uad6c", "unit": "celcius", "temperature": "mist", "humidity": 22, "description": 94}'

## LLM 요청 흐름

In [41]:
# LLM 에게 각 Function 들에 대해서 설명해주는 메타정보 'tools' 를 알려줘야 함.
tools = [{
    "type"        : "function",
    "function"    : {
        "name"        : "get_current_weather",
        "description" : "주어진 도시에 대해 현재 날씨 정보를 가져온다.\nGet current temperature for a given location.",
        "parameters"  : {
            "type"      : "object",
            "properties": {
                "city" : { "type": "string", "description": "날씨 정보를 얻고자하는 도시의 이름(영문).\nCity and country(english) e.g. Bogotá, Colombia" },
                "units": { "type": "string", "enum": ["metric", "imperial"], "description":"날씨 정보에 대한 온도 단위 설정. 섭씨를 사용하는 경우 metric을, 화씨를 사용하는 경우 imperial을 선택한다." }
            },
            "required": [
                "city"
            ],
            "additionalProperties": False
        }
    },

}]
messages = [
    {"role":"system", "content":"당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공하세요."},
    {"role":"user"  , "content":"서울 의 오늘 날씨를 알려줘."}
]

In [33]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)
response

ChatCompletion(id='chatcmpl-BmXENEKwMJTjTglYyLHHLk2TsBkm8', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9ZXSk3rBjlWB0mNrUGE4rg8b', function=Function(arguments='{"city":"Seoul, South Korea","units":"metric"}', name='get_current_weather'), type='function')]))], created=1750907183, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=23, prompt_tokens=159, total_tokens=182, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [36]:
# LLM  으로부터 받은 Function call 정보를 받을 수 있음
response_message = response.choices[0].message
print("response_message : ", response_message)
tool_calls = response_message.tool_calls
tool_calls

response_message :  ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9ZXSk3rBjlWB0mNrUGE4rg8b', function=Function(arguments='{"city":"Seoul, South Korea","units":"metric"}', name='get_current_weather'), type='function')])


[ChatCompletionMessageToolCall(id='call_9ZXSk3rBjlWB0mNrUGE4rg8b', function=Function(arguments='{"city":"Seoul, South Korea","units":"metric"}', name='get_current_weather'), type='function')]

In [42]:
# 함수 목록에서 해당함수를 찾고 실행 - 실행결과를 다시 LLM 에 질의

# 실행가능한 함수 목록
available_functions = {
    "get_current_weather" : get_current_weather
}

# LLM (assistant) : function_call 응답
messages.append(response_message)

for tool_call in tool_calls:
  function_name = tool_call.function.name
  func_to_call  = available_functions[function_name]

  func_args     = json.loads(tool_call.function.arguments)   # json string 을 python dict 로 변환

  print(func_to_call, "\n")
  print(func_args)

  func_response = func_to_call(**func_args)
  print(func_response)

  messages.append({"role":"tool", "tool_call_id":tool_call.id, "name":function_name, "content":func_response})

messages

<function get_current_weather at 0x7f01b817afc0> 

{'city': 'Seoul, South Korea', 'units': 'metric'}
{"location": "Seoul, South Korea", "unit": "celcius", "temperature": "broken clouds", "humidity": 23.76, "description": 94}


[{'role': 'system', 'content': '당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공하세요.'},
 {'role': 'user', 'content': '서울 의 오늘 날씨를 알려줘.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_9ZXSk3rBjlWB0mNrUGE4rg8b', function=Function(arguments='{"city":"Seoul, South Korea","units":"metric"}', name='get_current_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_9ZXSk3rBjlWB0mNrUGE4rg8b',
  'name': 'get_current_weather',
  'content': '{"location": "Seoul, South Korea", "unit": "celcius", "temperature": "broken clouds", "humidity": 23.76, "description": 94}'}]

In [43]:
# 최종LLM 응답
stream = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    stream=True
)
for chunk in stream:
  content = chunk.choices[0].delta.content
  if content is not None:
    print(content, end="")

오늘 서울의 날씨는 흐림으로, 기온은 약 23.8도입니다. 습도는 94%로 다소 습한 편입니다. 외출 시 우산을 챙기시는 것이 좋겠습니다!

In [45]:
# tool_call 을 포함한 함수
def run_conversation(prompt):
  messages = [
      {"role":"system", "content":"당신은 친절한 챗봇으로, 사용자의 요청에 답변을 제공하세요."},
      {"role":"user"  , "content":prompt}
  ]

  # 첫 번째 LLM 요청
  response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
      tools=tools,
  )

  # 첫 번째 LLM 응답 확인
  response_message = response.choices[0].message
  tool_calls       = response_message.tool_calls

  if tool_calls:
    # n 건의 함수 호출( 툴콜 개수 만큼 )
    messages.append(response_message)
    for tool_call in tool_calls:
      func_name     = tool_call.function.name
      func_to_call  = available_functions[func_name]
      func_args     = json.loads(tool_call.function.arguments)   # json string 을 python dict 로 변환

      func_response = func_to_call(**func_args)

      messages.append({"role":"tool", "tool_call_id":tool_call.id, "name":func_name, "content":func_response})

    # 두 번째 LLM 요청
    response = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=messages,
    )
  return response.choices[0].message.content

print("오늘 기분이 어때?\n", run_conversation("오늘 기분이 어때?"), "\n")
print("오늘 날짜와 오늘의 브라질 날씨 알려줘\n", run_conversation("오늘 날짜와 오늘의 브라질 날씨 알려줘"))

오늘 기분이 어때?
 저는 항상 기분이 좋습니다! 당신을 도와드릴 준비가 되어 있어요. 어떤 도움이 필요하신가요? 

오늘 날짜와 오늘의 브라질 날씨 알려줘
 오늘 날짜는 2023년 10월 8일입니다.

브라질의 날씨는 전반적으로 맑고, 다음과 같은 세부 정보가 있습니다:

- **브라질 전체**: 맑은 하늘, 기온은 약 24.86도, 습도는 90%입니다.
- **리우데자네이루**: 맑은 하늘, 기온은 약 18.9도, 습도는 68%입니다.
- **상파울루**: 맑은 하늘, 기온은 약 13.38도, 습도는 84%입니다.

다른 도움이 필요하시면 말씀해 주세요!
